In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# 손실 함수

In [ ]:
# 오차제곱합
def sum_squares_error(y, t): 
  return 0.5 * np.sum((y-t)**2)

# 원 핫 인코딩
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] # 정답 2

# 신경망의 출력 2인 경우
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(sum_squares_error(np.array(y), np.array(t)))

# 신경망의 출력이 7인 경우
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(sum_squares_error(np.array(y), np.array(t)))

0.09750000000000003
0.5975


In [ ]:
# 크로스 엔트로피
def cross_entropy_error(y, t):
  delta = 1e-7 # log에 0 이 입력되는 사태 방지
  return -np.sum(t * np.log(y + delta)) 

t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

print(cross_entropy_error(np.array(y), np.array(t)))

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

print(cross_entropy_error(np.array(y), np.array(t)))

0.510825457099338
2.302584092994546


In [ ]:
path = '/content/drive/MyDrive/deep-learning-from-scratch-master'
os.chdir(path)
sys.path.append(os.chdir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape, '\n')

(60000, 784)
(60000, 10) 



In [ ]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)

#x_batch = x_train[batch_mask]
#t_batch = t_train[batch_mask]
print(batch_mask)
print(np.random.choice(60000, 10))

[52950 16350 22210 19322 26941 10697 44587 30966 34557 31363]
[ 1686 26428  9950 24176 53650 54113 14592 11697   324 17053]
